In [1]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from OrbitalRotation.tools.orbital_rotation_general import orbital_rotation_p_q, orbital_rotation_p_q_nf
from OrbitalRotation.tools.obt_to_tbt import get_obt_combined_tbt
import scipy
import pyscf
from basic_definitions import chemist_to_physicist, chemist_to_physicist_trunc
import dmrghandler.src.dmrghandler.dmrg_calc_prepare as dmrg_calc_prepare
from dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
from dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc
from get_dmrg_config import get_dmrg_process_param, get_dmrg_config, get_dmrg_config_type1, get_custom_bd_dmrg_config

In [58]:
import shutil
import os

# Define the directory path
dir_path = "./mps_directory"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")
    
# Define the directory path
dir_path = "./mps_directory2"

# Check if the directory exists
if os.path.exists(dir_path):
    # Delete the directory and all its contents
    shutil.rmtree(dir_path)
    print(f"Deleted '{dir_path}' and all its contents.")
else:
    print(f"Directory '{dir_path}' does not exist.")

Deleted './mps_directory' and all its contents.
Directory './mps_directory2' does not exist.


In [59]:
from pyscf import gto, scf

mol = gto.M(atom="O 0 0 0; H  0 1 0; H 0 0 1", basis="sto3g", verbose=0)
mf = scf.RHF(mol).run(conv_tol=1E-14)
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=1)
print(f"n Cas: {ncas}")
print(f"orb sym: {orb_sym}")
driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

idx = driver.orbital_reordering(h1e, g2e)
h1e = h1e[idx][:, idx]
g2e = g2e[idx][:, idx][:, :, idx][:, :, :, idx]
orb_sym = np.array(orb_sym)[idx]

absorbed_tbt = pyscf.fci.direct_nosym.absorb_h1e(np.array(h1e.copy()), np.array(g2e.copy()), norb=ncas, nelec=n_elec)
h_in_tbt = pyscf.ao2mo.restore(1, absorbed_tbt.copy(), ncas).astype(np.array(h1e.copy()).dtype, copy=False)
combined_tbt_optimizing = 0.5 * h_in_tbt
obt_dummy = np.zeros((ncas, ncas))
obt, tbt = chemist_to_physicist(obt_dummy, 0.5 * h_in_tbt)

n Cas: 7
orb sym: [0, 0, 0, 0, 0, 0, 0]


In [60]:
dmrg_params = get_dmrg_config(
    ncas, n_elec, spin,
    2 * spin + 1)
loop_results = dmrg_central_loop(obt, tbt, dmrg_params, 1000, 1e6, 1e-8   , "./mps_directory", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       4672

Build MPO | Nsites =     7 | Nterms =       4672 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2241 SPT = 0.8276 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      591 SPT = 0.9259 Tmvc = 0.000 T = 0.002
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.016 Tmvc-total = 0.002 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4153 SIZE =        33216 SPT = 0.8750

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4672

Build MPO | Nsites =     7 | Nterms =       4672 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2241 SPT = 0.8276 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      591 SPT = 0.9259 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.011 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4153 SIZE =        33216 SPT = 0.8750

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4672

Build MPO | Nsites =     7 | Nterms =       4672 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2241 SPT = 0.8276 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      591 SPT = 0.9259 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.011 Tmvc-total = 0.001 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4153 SIZE =        33216 SPT = 0.8750

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       4672

Build MPO | Nsites =     7 | Nterms =       4672 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.003
 Site =     1 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =      599 SPT = 0.9249 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =   114 DW = 0.00e+00 NNZ =     2241 SPT = 0.8276 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      591 SPT = 0.9259 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    30 DW = 0.00e+00 NNZ =      331 SPT = 0.8424 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       30 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.013 Tmvc-total = 0.002 MPO bond dimension =   114 MaxDW = 0.00e+00
NNZ =         4153 SIZE =        33216 SPT = 0.8750

In [47]:
def get_H_V(two_bt):
    n = two_bt.shape[0]
    n_2 = n // 2
    H = np.zeros((n, n, n, n))
    V = np.zeros((n, n, n, n))
    for p in range(n):
        for q in range(n):
            for r in range(n):
                for s in range(n):
                    if p < n_2 and q < n_2 and r < n_2 and s < n_2:
                        H[p, q, r, s] = two_bt[p, q, r, s]
                    elif p >= n_2 and q >= n_2 and r >= n_2 and s >= n_2:
                        H[p, q, r, s] = two_bt[p, q, r, s]
                    else:
                        V[p, q, r, s] = two_bt[p, q, r, s]
    
    return H, V

In [61]:
H, V = get_H_V(combined_tbt_optimizing)
obt_dummy = np.zeros((ncas, ncas))
# obt, tbt = chemist_to_physicist_trunc(obt_dummy, H, 1e-15)

In [62]:
print(H[4, 4, 1, 0])

0.0


In [63]:
n = obt_dummy.shape[0]
obt_phy = obt_dummy.copy()
tbt_copy = H.copy()
for p in range(n):
    for q in range(n):
        obt_phy[p, q] += sum([tbt.copy()[p, r, r, q] for r in range(n)])
        print(f"At {p, q} : {obt_phy[p, q]}")
for p in range(n):
    for q in range(n):
        if abs(obt_phy[p, q]) < 1e-15:
            obt_phy[p, q] = 0
        print(f"At {p, q} : {obt_phy[p, q]}")
        for r in range(n):
            for s in range(n):
                if abs(tbt_copy[p, q, r, s]) < 1e-15:
                    tbt_copy[p, q, r, s] = 0



At (0, 0) : -0.28739106628253824
At (0, 1) : -0.16498093916456827
At (0, 2) : -2.0480616281985143e-15
At (0, 3) : -2.0643720433645834e-16
At (0, 4) : 8.080188711544163e-17
At (0, 5) : 2.451388377504152e-17
At (0, 6) : -2.338782719382669e-16
At (1, 0) : -0.16498093916456857
At (1, 1) : -0.1530051881517401
At (1, 2) : -1.995352768106069e-16
At (1, 3) : -1.9323364264225517e-15
At (1, 4) : -2.348056438198228e-17
At (1, 5) : 8.560694513594829e-16
At (1, 6) : 1.738085974327054e-15
At (2, 0) : -2.2698796615525707e-15
At (2, 1) : -1.0810918073537184e-16
At (2, 2) : -0.16197701824415497
At (2, 3) : 0.13692456842816225
At (2, 4) : -1.3414391760844076e-17
At (2, 5) : -0.0519157643085334
At (2, 6) : -0.1104080922217289
At (3, 0) : -4.1967783143727587e-16
At (3, 1) : -1.951664366113187e-15
At (3, 2) : 0.13692456842816217
At (3, 3) : -0.3168009949628767
At (3, 4) : -8.536565565696178e-17
At (3, 5) : 0.005468842401734723
At (3, 6) : -0.09418361409943124
At (4, 0) : 8.080188711544169e-17
At (4, 1) : -

In [64]:
print(tbt_copy[4, 4, 1, 0])

0.0


In [65]:
dmrg_params = get_dmrg_config(
    ncas, n_elec, spin,
    2 * spin + 1)
loop_results = dmrg_central_loop(obt_phy, 2 * tbt_copy, dmrg_params, 1000, 1e6, 1e-8   , "./mps_directory2", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =        496

Build MPO | Nsites =     7 | Nterms =        496 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    22 DW = 0.00e+00 NNZ =       22 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       77 SPT = 0.8654 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =    10 DW = 0.00e+00 NNZ =       31 SPT = 0.8808 Tmvc = 0.000 T = 0.001
 Site =     3 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =       43 SPT = 0.8346 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    38 DW = 0.00e+00 NNZ =       67 SPT = 0.9322 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      163 SPT = 0.8350 Tmvc = 0.000 T = 0.002
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.009 Tmvc-total = 0.001 MPO bond dimension =    38 MaxDW = 0.00e+00
NNZ =          429 SIZE =         3116 SPT = 0.8623

In [66]:
n = obt_dummy.shape[0]
obt_phy = obt_dummy.copy()
tbt_copy = V.copy()
for p in range(n):
    for q in range(n):
        obt_phy[p, q] += sum([tbt.copy()[p, r, r, q] for r in range(n)])
        print(f"At {p, q} : {obt_phy[p, q]}")
for p in range(n):
    for q in range(n):
        if abs(obt_phy[p, q]) < 1e-15:
            obt_phy[p, q] = 0
        print(f"At {p, q} : {obt_phy[p, q]}")
        for r in range(n):
            for s in range(n):
                if abs(tbt_copy[p, q, r, s]) < 1e-15:
                    tbt_copy[p, q, r, s] = 0

At (0, 0) : -0.28739106628253824
At (0, 1) : -0.16498093916456827
At (0, 2) : -2.0480616281985143e-15
At (0, 3) : -2.0643720433645834e-16
At (0, 4) : 8.080188711544163e-17
At (0, 5) : 2.451388377504152e-17
At (0, 6) : -2.338782719382669e-16
At (1, 0) : -0.16498093916456857
At (1, 1) : -0.1530051881517401
At (1, 2) : -1.995352768106069e-16
At (1, 3) : -1.9323364264225517e-15
At (1, 4) : -2.348056438198228e-17
At (1, 5) : 8.560694513594829e-16
At (1, 6) : 1.738085974327054e-15
At (2, 0) : -2.2698796615525707e-15
At (2, 1) : -1.0810918073537184e-16
At (2, 2) : -0.16197701824415497
At (2, 3) : 0.13692456842816225
At (2, 4) : -1.3414391760844076e-17
At (2, 5) : -0.0519157643085334
At (2, 6) : -0.1104080922217289
At (3, 0) : -4.1967783143727587e-16
At (3, 1) : -1.951664366113187e-15
At (3, 2) : 0.13692456842816217
At (3, 3) : -0.3168009949628767
At (3, 4) : -8.536565565696178e-17
At (3, 5) : 0.005468842401734723
At (3, 6) : -0.09418361409943124
At (4, 0) : 8.080188711544169e-17
At (4, 1) : -

In [67]:
dmrg_params = get_dmrg_config(
    ncas, n_elec, spin,
    2 * spin + 1)
loop_results = dmrg_central_loop(obt_phy, 2 * tbt_copy, dmrg_params, 1000, 1e6, 1e-8   , "./mps_directory3", verbosity=1, move_mps_to_final_storage_path=None)

integral cutoff error =  0.0
mpo terms =       1290

Build MPO | Nsites =     7 | Nterms =       1290 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    17 DW = 0.00e+00 NNZ =       17 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    57 DW = 0.00e+00 NNZ =       53 SPT = 0.9453 Tmvc = 0.000 T = 0.002
 Site =     2 /     7 .. Mmpo =    76 DW = 0.00e+00 NNZ =      357 SPT = 0.9176 Tmvc = 0.000 T = 0.002
 Site =     3 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      727 SPT = 0.8633 Tmvc = 0.000 T = 0.003
 Site =     4 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =       91 SPT = 0.9776 Tmvc = 0.000 T = 0.002
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      107 SPT = 0.9290 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.001
Ttotal =      0.012 Tmvc-total = 0.001 MPO bond dimension =    76 MaxDW = 0.00e+00
NNZ =         1378 SIZE =        16232 SPT = 0.9151

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1290

Build MPO | Nsites =     7 | Nterms =       1290 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    17 DW = 0.00e+00 NNZ =       17 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    57 DW = 0.00e+00 NNZ =       53 SPT = 0.9453 Tmvc = 0.000 T = 0.001
 Site =     2 /     7 .. Mmpo =    76 DW = 0.00e+00 NNZ =      357 SPT = 0.9176 Tmvc = 0.000 T = 0.001
 Site =     3 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      727 SPT = 0.8633 Tmvc = 0.000 T = 0.001
 Site =     4 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =       91 SPT = 0.9776 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      107 SPT = 0.9290 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.006 Tmvc-total = 0.001 MPO bond dimension =    76 MaxDW = 0.00e+00
NNZ =         1378 SIZE =        16232 SPT = 0.9151

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1290

Build MPO | Nsites =     7 | Nterms =       1290 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    17 DW = 0.00e+00 NNZ =       17 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    57 DW = 0.00e+00 NNZ =       53 SPT = 0.9453 Tmvc = 0.000 T = 0.001
 Site =     2 /     7 .. Mmpo =    76 DW = 0.00e+00 NNZ =      357 SPT = 0.9176 Tmvc = 0.000 T = 0.001
 Site =     3 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      727 SPT = 0.8633 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =       91 SPT = 0.9776 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      107 SPT = 0.9290 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.006 Tmvc-total = 0.001 MPO bond dimension =    76 MaxDW = 0.00e+00
NNZ =         1378 SIZE =        16232 SPT = 0.9151

...

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1290

Build MPO | Nsites =     7 | Nterms =       1290 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    17 DW = 0.00e+00 NNZ =       17 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    57 DW = 0.00e+00 NNZ =       53 SPT = 0.9453 Tmvc = 0.000 T = 0.001
 Site =     2 /     7 .. Mmpo =    76 DW = 0.00e+00 NNZ =      357 SPT = 0.9176 Tmvc = 0.000 T = 0.001
 Site =     3 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      727 SPT = 0.8633 Tmvc = 0.000 T = 0.001
 Site =     4 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =       91 SPT = 0.9776 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      107 SPT = 0.9290 Tmvc = 0.000 T = 0.000
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.005 Tmvc-total = 0.001 MPO bond dimension =    76 MaxDW = 0.00e+00
NNZ =         1378 SIZE =        16232 SPT = 0.9151

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm


integral cutoff error =  0.0
mpo terms =       1290

Build MPO | Nsites =     7 | Nterms =       1290 | Algorithm = FastBIP | Cutoff = 1.00e-120
 Site =     0 /     7 .. Mmpo =    17 DW = 0.00e+00 NNZ =       17 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     7 .. Mmpo =    57 DW = 0.00e+00 NNZ =       53 SPT = 0.9453 Tmvc = 0.000 T = 0.001
 Site =     2 /     7 .. Mmpo =    76 DW = 0.00e+00 NNZ =      357 SPT = 0.9176 Tmvc = 0.000 T = 0.001
 Site =     3 /     7 .. Mmpo =    70 DW = 0.00e+00 NNZ =      727 SPT = 0.8633 Tmvc = 0.000 T = 0.002
 Site =     4 /     7 .. Mmpo =    58 DW = 0.00e+00 NNZ =       91 SPT = 0.9776 Tmvc = 0.000 T = 0.001
 Site =     5 /     7 .. Mmpo =    26 DW = 0.00e+00 NNZ =      107 SPT = 0.9290 Tmvc = 0.000 T = 0.001
 Site =     6 /     7 .. Mmpo =     1 DW = 0.00e+00 NNZ =       26 SPT = 0.0000 Tmvc = 0.000 T = 0.000
Ttotal =      0.006 Tmvc-total = 0.001 MPO bond dimension =    76 MaxDW = 0.00e+00
NNZ =         1378 SIZE =        16232 SPT = 0.9151

/Users/rick/PycharmProjects/croor/dmrghandler/src/dmrghandler/energy_extrapolation.py:344: RuntimeWarning: divide by zero encountered in log
  a = np.log(alpha)  # α = exp(a), np.log is the natural logarithm
